In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import numpy as np
import torchsummary
import os
from PIL import Image

torch.cuda.get_device_name(0)

In [29]:
!pip install tqdm

  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)



[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
# 모델정의
class ResNet50Classifier(nn.Module):
    def __init__(self, num_classes=4800):
        super(ResNet50Classifier, self).__init__()
        
        # Resnet50 model
        # pretrained 모델 사용
        self.backborn = models.resnet50(pretrained=True)
        
        for param in self.backborn.parameters():
            param.requires_grad = False
        
        # resnet50 출력채널수
        num_features = self.backborn.fc.out_features
        
        # 우리가 분류할 class만큼 full connected 레이어 추가
        self.classifier = nn.Linear(num_features, num_classes)
    
    def forward(self, x):
        x = self.backborn(x)
        x = self.classifier(x)
        return x

In [25]:
# 이미지 투명도 제거 Transform
class RemoveAlpha:
    def __call__(self, img):
        img = img.convert('RGB')
        return img

batch_size = 16
    
# 데이터 로더
base_path = './sample_data'

# 데이터셋 전처리
transform = transforms.Compose([
    RemoveAlpha(),
    transforms.CenterCrop(1200),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    # transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
])

image_dataset = ImageFolder(base_path, transform=transform)

val_size = int(len(image_dataset) * 0.2)
train_size = len(image_dataset) - val_size

train_dataset, val_dataset = torch.utils.data.random_split(image_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [26]:
# 학습함수
def train(model, train_loader, valid_loader, criterion, optimizer, device, epochs):
    model.to(device)
    
    # epochs 만큼 반복
    for epoch in range(epochs):
        # 캐시 비우기
        torch.cuda.empty_cache()
        # train모드
        model.train()
        # train 데이터 가져옴
        for images, labels in train_loader:
            # 장치로 보냄
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            
            # loss 계산
            loss = criterion(outputs, labels)
            
            # backpropagation
            loss.backward()
            optimizer.step()
            del images, labels
        # validation
        valid_loss = 0.0
        valid_accuracy = 0.0
        # eval모드
        model.eval()
        with torch.no_grad():
            for images, labels in valid_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                
                _, pred = torch.max(outputs, 1)
                accuracy = torch.sum(pred == labels.data)
                
                valid_loss += loss.item() * images.size(0)
                valid_accuracy += accuracy.item()
        valid_loss /= len(valid_loader.dataset)
        valid_accuracy /= len(valid_loader.dataset)
        
        print(f'Epoch {epoch+1}/{epochs} : loss : {loss.item():.3f}, valid_loss : {valid_loss:.3f}, valid_accuracy : {valid_accuracy:.3f}') 

In [28]:
model = ResNet50Classifier(num_classes=20)
# train_loader = 
# valid_loader = None
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
epochs = 12

train(model, train_loader, val_loader, criterion, optimizer, device, epochs)

Epoch 1/12 : loss : 0.274, valid_loss : 0.274, valid_accuracy : 0.975
Epoch 2/12 : loss : 0.118, valid_loss : 0.118, valid_accuracy : 0.994
Epoch 3/12 : loss : 0.052, valid_loss : 0.052, valid_accuracy : 1.000
Epoch 4/12 : loss : 0.096, valid_loss : 0.096, valid_accuracy : 1.000
Epoch 5/12 : loss : 0.046, valid_loss : 0.046, valid_accuracy : 1.000
Epoch 6/12 : loss : 0.017, valid_loss : 0.017, valid_accuracy : 1.000
Epoch 7/12 : loss : 0.028, valid_loss : 0.028, valid_accuracy : 1.000
Epoch 8/12 : loss : 0.005, valid_loss : 0.005, valid_accuracy : 1.000
Epoch 9/12 : loss : 0.010, valid_loss : 0.010, valid_accuracy : 1.000


KeyboardInterrupt: 

In [27]:
# 메모리 정리
del model
torch.cuda.empty_cache()
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      | 429085 KiB |   4969 MiB |   1735 GiB |   1734 GiB |
|       from large pool | 358144 KiB |   4812 MiB |   1716 GiB |   1715 GiB |
|       from small pool |  70941 KiB |    209 MiB |     19 GiB |     19 GiB |
|---------------------------------------------------------------------------|
| Active memory         | 429085 KiB |   4969 MiB |   1735 GiB |   1734 GiB |
|       from large pool | 358144 KiB |   4812 MiB |   1716 GiB |